In [1]:
#Installing packages
import pymongo
import pandas as pd
import json

In [2]:
#Connecting to mongodb
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client.database1

In [3]:
#Loading in into the data
collection = db.collection1

In [4]:
total_docs = collection.count_documents({})
total_docs

3773

In [5]:
#MongoDB test sample
query = {
    "Job Category": {
        "$regex": 'Health'
    }    
}
cur = collection.find(query)

In [6]:
#Appending the result into the list
a  = []

for i in cur:
    a.append(i)

In [7]:
#Visualizing the data
dat = pd.DataFrame(a)
dat = dat.iloc[:,1:61]
dat.head()

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Classification,Title Code No,Level,Job Category,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,424339,DEPT OF HEALTH/MENTAL HYGIENE,External,1,"Triage Nurse, Bureau of Public Health Clinics",PUBLIC HEALTH NURSE,Competitive-1,51011,3,Health,...,**IMPORTANT NOTES TO ALL CANDIDATES: Please ...,Apply online with a cover letter to https://a1...,,,,New York City Residency is not required for th...,03/01/2022,,04/05/2022,04/12/2022
1,526663,DEPT OF HEALTH/MENTAL HYGIENE,Internal,1,"College Aide, Bureau of Vital Statistics",COLLEGE AIDE (ALL CITY DEPTS),Non-Competitive-5,10209,1,Health,...,**IMPORTANT NOTES TO ALL CANDIDATES: Please n...,Apply online with a cover letter to https://a1...,35 hours/week when school is not in session,,,New York City residency is generally required ...,04/07/2022,05-AUG-2022,04/07/2022,04/12/2022
2,474070,DEPT OF HEALTH/MENTAL HYGIENE,External,1,"Race to Justice Data Analyst, Office of First ...",CITY RESEARCH SCIENTIST,Non-Competitive-5,21744,2,"Health Policy, Research & Analysis",...,**IMPORTANT NOTES TO ALL CANDIDATES: Please n...,Apply online with a cover letter to https://a1...,,,,New York City residency is generally required ...,02/03/2022,30-APR-2022,03/30/2022,04/12/2022
3,520157,NYC POLICE PENSION FUND,External,1,Medical Board Physician,PHYSICIAN,Non-Competitive-5,53010,0,Health,...,Class of Position: Independent Contractor Sala...,Send a CV and cover letter in a PDF format to ...,Minimum hours: 4/week,"1 Lefrak City Plaza, Corona, NY",,New York City Residency is not required for th...,02/10/2022,,02/10/2022,04/12/2022
4,460190,DEPT OF HEALTH/MENTAL HYGIENE,Internal,1,NYCTC Evidence-Based Program Coordinator,COMMUNITY COORDINATOR,Non-Competitive-5,56058,0,Constituent Services & Community Programs Health,...,**IMPORTANT NOTES TO ALL CANDIDATES: Please n...,Apply online with a cover letter to https://a1...,,,,New York City residency is generally required ...,01/21/2022,21-MAY-2022,02/23/2022,04/12/2022


In [8]:
#ApacheSpark
!pip install pyspark

In [9]:
#Connecting pyspark
from pyspark.sql.types import *
from datetime import datetime

from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
from pyspark.sql.types import BooleanType
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

In [10]:
sc = SparkContext() 
config = sc.getConf()
config.set('spark.cores.max','4')
config.set('spark.executor.memory', '8G')
config.set('spark.driver.maxResultSize', '8g')
config.set('spark.kryoserializer.buffer.max', '512m')
config.set("spark.driver.cores", "4")
sc.stop()

sc = SparkContext(conf = config) 
sqlContext = SQLContext(sc)
    # print("Using Apache Spark Version", sc.version)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/28 20:03:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/28 20:03:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/28 20:03:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/04/28 20:03:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/28 20:03:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
/Users/yuanhongxie/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [11]:
#Functions to convert pandas dataframe to spark dataframe

def equivalent_type(f):
      if f == 'datetime64[ns]': return TimestampType()
      elif f == 'int64': return LongType()
      elif f == 'int32': return IntegerType()
      elif f == 'float64': return FloatType()
      else: return StringType()
      
def define_structure(string, format_type):
      try: typo = equivalent_type(format_type)
      except: typo = StringType()
      return StructField(string, typo)
      
def pandas_to_spark(pandas_df):
      columns = list(pandas_df.columns)
      types = list(pandas_df.dtypes)
      struct_list = []
      for column, typo in zip(columns, types): 
        struct_list.append(define_structure(column, typo))
      p_schema = StructType(struct_list)
      return sqlContext.createDataFrame(pandas_df, p_schema)

In [12]:
spark_df = pandas_to_spark(dat)

In [13]:
spark_df.columns

['Job ID',
 'Agency',
 'Posting Type',
 '# Of Positions',
 'Business Title',
 'Civil Service Title',
 'Title Classification',
 'Title Code No',
 'Level',
 'Job Category',
 'Full-Time/Part-Time indicator',
 'Career Level',
 'Salary Range From',
 'Salary Range To',
 'Salary Frequency',
 'Work Location',
 'Division/Work Unit',
 'Job Description',
 'Minimum Qual Requirements',
 'Preferred Skills',
 'Additional Information',
 'To Apply',
 'Hours/Shift',
 'Work Location 1',
 'Recruitment Contact',
 'Residency Requirement',
 'Posting Date',
 'Post Until',
 'Posting Updated',
 'Process Date']

In [14]:
#test
spark_df.show(5)

22/04/28 20:03:51 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+--------------------+------------+--------------+--------------------+--------------------+--------------------+-------------+-----+--------------------+-----------------------------+--------------------+-----------------+---------------+----------------+--------------------+--------------------+--------------------+-------------------------+--------------------+----------------------+--------------------+--------------------+--------------------+-------------------+---------------------+------------+-----------+---------------+------------+
|Job ID|              Agency|Posting Type|# Of Positions|      Business Title| Civil Service Title|Title Classification|Title Code No|Level|        Job Category|Full-Time/Part-Time indicator|        Career Level|Salary Range From|Salary Range To|Salary Frequency|       Work Location|  Division/Work Unit|     Job Description|Minimum Qual Requirements|    Preferred Skills|Additional Information|            To Apply|         Hours/Shift|     W

In [15]:
start_time = datetime.now()
from flask import Flask, request, render_template
app = Flask("ETL-Pipeline")

@app.route('/')
def my_form():
    return render_template('my-form.html')

@app.route('/', methods=['POST'])
def my_form_post():
    val = request.form['userinput']
    query = {
    "Job Category": {
        "$regex": val} }
    cur = collection.find(query)
    a  = []
    for i in cur:
        a.append(i)
    dat = pd.DataFrame(a)
    dat = dat.iloc[:,1:61]
    spark_df = pandas_to_spark(dat)
    cb_rdd = spark_df.select('*').rdd.map(lambda row: [str(row[i]) for i in ['Agency']])
    cb_sdf2 = sqlContext.createDataFrame(cb_rdd,['Agency'])
    final_df = cb_sdf2.toPandas(),
   
    
    return str(final_df) #displaying results on the webpage
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:00.395498


In [ ]:
app.run(host='localhost', port=5002)

 * Serving Flask app "ETL-Pipeline" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Apr/2022 20:03:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2022 20:04:05] "POST / HTTP/1.1" 200 -                    
127.0.0.1 - - [28/Apr/2022 20:04:26] "GET / HTTP/1.1" 200 -
